<h2>Funwave-TVD</h2>
Funwave-TVD is a code to simulate the shallow water and Boussinesq equations written by Dr. Fengyan Shi. It is capable of running a variety of complex ocean simulations. For more information, see Dr. Fengyan Shi's website: https://fengyanshi.github.io/shi_website/_build/html/funwave.html

We will use it as our example scientific code for this tutorial because it is reasonably complex, real scientific code but can be compiled and run quickly.

In [ ]:
!mkdir -p ~/agave

%cd ~/agave

!pip3 install --upgrade setvar

import re
import os
import sys
from setvar import *
from time import sleep

# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt


The parameter file for Funwave-TVD is called "input.txt". Here we create it using writefile. You don't need to understand the details of how this file for this tutorial.

In [ ]:
writefile("input.txt","""
!INPUT FILE FOR FUNWAVE_TVD
  ! NOTE: all input parameter are capital sensitive
  ! --------------------TITLE-------------------------------------
  ! title only for log file
TITLE = VESSEL
  ! -------------------HOT START---------------------------------
HOT_START = F
FileNumber_HOTSTART = 1
  ! -------------------PARALLEL INFO-----------------------------
  ! 
  !    PX,PY - processor numbers in X and Y
  !    NOTE: make sure consistency with mpirun -np n (px*py)
  !    
PX = 4
PY = 1
  ! --------------------DEPTH-------------------------------------
  ! Depth types, DEPTH_TYPE=DATA: from depth file
  !              DEPTH_TYPE=FLAT: idealized flat, need depth_flat
  !              DEPTH_TYPE=SLOPE: idealized slope, 
  !                                 need slope,SLP starting point, Xslp
  !                                 and depth_flat
DEPTH_TYPE = FLAT
DEPTH_FLAT = 10.0
  ! -------------------PRINT---------------------------------
  ! PRINT*,
  ! result folder
RESULT_FOLDER = output/

  ! ------------------DIMENSION-----------------------------
  ! global grid dimension
Mglob = 500
Nglob = 100

  ! ----------------- TIME----------------------------------
  ! time: total computational time/ plot time / screen interval 
  ! all in seconds
TOTAL_TIME = 10.0
PLOT_INTV = 1.0
PLOT_INTV_STATION = 50000.0
SCREEN_INTV = 1.0
HOTSTART_INTV = 360000000000.0

WAVEMAKER = INI_GAU
AMP = 3.0
Xc = 250.0
Yc = 50.0
WID = 20.0

  ! -----------------GRID----------------------------------
  ! if use spherical grid, in decimal degrees
  ! cartesian grid sizes
DX = 1.0
DY = 1.0
  ! ----------------SHIP WAKES ----------------------------
VESSEL_FOLDER = ./
NumVessel = 2
  ! -----------------OUTPUT-----------------------------
ETA = T
U = T
V = T
""")

Here is a simple bash script to run the code....

In [ ]:
writefile("run.sh","""
#!/bin/bash
export LD_LIBRARY_PATH=/usr/local/lib
mkdir -p rundir
cd ./rundir
cp ../input.txt .
mpirun -np 4 ~/FUNWAVE-TVD/src/funwave_vessel
""")

Here's how the average developer might run the code when doing their research.

In [ ]:
!scp -o "StrictHostKeyChecking=no" -P 10022 input.txt run.sh $VM_IPADDRESS:.
!ssh -p 10022 $VM_IPADDRESS bash run.sh

Here's how they might retrieve the data.

In [ ]:
!scp -q -r -P 10022 $VM_IPADDRESS:rundir/output .

In [ ]:
!ls output

Now let's use Python to graph the output

In [ ]:
data = np.genfromtxt("output/v_00003")
fig = plt.figure(figsize=(12,12))
pltres = plt.imshow(data[::-1,:])
plt.show()